In [ ]:
from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

from google.cloud import storage
import io

import torchvision.models as models


from matplotlib import pylab as P
from torchvision import models, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

# Preprocessing

In [ ]:
client = storage.Client()
bucket = client.get_bucket('dl-cnn-data')

categories = ['flower', 'apple', 'baseball', 'basketball', 'bird', 'book', 'bus', 'car', 'cat', 'dog']
label_dict = {0:'flower', 1:'apple', 2:'baseball', 3:'basketball', 4:'bird',
                      5:'book',6:'bus', 7:'car', 8:'cat', 9:'dog'}
# Data Hyperparameters
test_set_split = 0.2
max_items_per_class = 10000
verbose = True
IMAGE_SIZE = 28

In [ ]:
is_write = False

data = pd.DataFrame()

for idx, category in label_dict.items():
    if verbose:
        print (idx, category)
        
    blob = bucket.get_blob('raw_data/' + category + '.npy')
    with io.BytesIO() as in_memory_file:
        blob.download_to_file(in_memory_file)
        in_memory_file.seek(0)
        category_data = np.load(in_memory_file)
        
    category_data = category_data[0:max_items_per_class, :]
    category_data = pd.DataFrame(category_data)
    category_data['label'] = idx
    
    data = pd.concat([data, category_data], axis=0, ignore_index=True)

data = data.loc[np.random.permutation(data.shape[0]), :]

if is_write:
    data.to_csv('cnn_data.csv')

In [ ]:
# split data
X_train, X_test, y_train, y_test = train_test_split(
    data.iloc[:, 0:-1], data.iloc[:, -1], 
    test_size=test_set_split, random_state=0)

X_train = torch.from_numpy(np.array(X_train)).float()
X_test = torch.from_numpy(np.array(X_test)).float()
y_train = torch.from_numpy(np.array(y_train)).long()
y_test = torch.from_numpy(np.array(y_test)).long()

def _preprocess_X(input_data):
    input_data = input_data.reshape(input_data.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)
    input_data = input_data.permute((0, 3, 1, 2))
    output = input_data / 255.
    return output

X_train = _preprocess_X(X_train)
X_test = _preprocess_X(X_test)


if torch.cuda.is_available():
    X_train, y_train = X_train.cuda(), y_train.cuda() 
    X_test, y_test = X_test.cuda(), y_test.cuda() 

# CNN and Resnet

In [ ]:
output_size = 10

dropout = 0.0
weight_decay = 0.0
# n_chunks = 700
# learning_rate = 0.001
optimizer = 'SGD'
epochs = 20

In [ ]:
def train(model, X_train, y_train, epochs = 10, n_chunks = 1000, learning_rate = 0.003, 
          weight_decay = 0, optimizer = 'SGD', is_time=False):
    
    print("Training model with epochs = {epochs}, learning rate = {lr}\n".format(epochs = epochs, lr = learning_rate))
    criterion = nn.CrossEntropyLoss()
    
    if optimizer == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer ==  'ADAM':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:
        print ("ERROR: optimizer undefined")
    
    losses = []
    res_train = []
    res_test = []
    res_time = []
    for epoch in range(epochs):
        print (epoch)
        
        if is_time:
            start_time = time.time()
        
        running_loss = 0
        images = torch.chunk(X_train, n_chunks)
        labels = torch.chunk(y_train, n_chunks)

        for i in range(len(images)):
            if (i%500 == 0):
                print ("processing ", i)
            optimizer.zero_grad()
            output = model.forward(images[i])
            loss = criterion(output, labels[i].squeeze())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
    
        losses.append(running_loss)
        print("Epoch: {}/{}... ".format(epoch + 1, epochs), "Loss: {:.4f}".format(running_loss))
        
        accuracy_train, accuracy_test = evaluate_model(model, X_train, y_train, X_test, y_test, n_samples_tested=10000000000)
        res_train.append(accuracy_train)
        res_test.append(accuracy_test)
        print ("accuracy", accuracy_train, accuracy_test)
        
        if is_time:
            time_taken = time.time() - start_time
            res_time.append(time_taken)
            print ('time taken', time_taken)
            
        res = losses, res_train, res_test
        if is_time:
            res = losses, res_train, res_test, res_time
        
    return res

def get_prediction_probability_distribution(model, input_data):
    with torch.no_grad():
        logits = model.forward(input_data)
    return torch.nn.functional.softmax(logits, dim=1)

def get_labels(model, input_data):
    probability_distribution = get_prediction_probability_distribution(model, input_data)
    pred_labels = torch.argmax(probability_distribution, axis=1, keepdims=True)
    return pred_labels

def evaluate_model(model, X_train, y_train, X_test, y_test, n_samples_tested = 5000):
    train_pred_labels = get_labels(model, X_train[:n_samples_tested])
    test_pred_labels = get_labels(model, X_test[:n_samples_tested])
    
    accuracy_train = accuracy_score(y_train[:n_samples_tested].cpu(), train_pred_labels.cpu())
    accuracy_test = accuracy_score(y_test[:n_samples_tested].cpu(), test_pred_labels.cpu())

    print("Accuracy score for train set is {} \n".format(accuracy_train))
    print("Accuracy score for test set is {} \n".format(accuracy_test))

    return accuracy_train, accuracy_test

In [ ]:
model_key_to_name = {
    0: 'cnn',
    1:'resnet18',
    1.5: 'resnet18_eval',
    2:'resnet34',
    3:'resnet50',
    4:'alexnet',
    5:'vgg16',
    6:'custom_resnet18',
}
model_key = 1

learning_rate_list = [0.05]

n_chunks_list = [2000]

kernel_sizes = [3, 11]

In [ ]:
# resnet18 = models.resnet18(pretrained=False)
# resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=k, stride=1, padding=1,bias=False)
# resnet18.fc = nn.Linear(in_features=512, out_features=10, bias=True)
# model = resnet18
# model

In [ ]:
models.resnet18(pretrained=False)

In [ ]:
for lr in learning_rate_list:
    for cur_chunks in n_chunks_list:
    # for k in kernel_sizes:
        # try:
        if model_key == 0:
            model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0)),
          ('relu2', nn.ReLU()),
          ('maxpool1', nn.MaxPool2d(2, stride=None)),
    
          ('conv3', nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0)),
          ('relu3', nn.ReLU()),
          ('conv4', nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)),
          ('relu4', nn.ReLU()),
          ('maxpool2', nn.MaxPool2d(2, stride=None)),
    
          ('conv5', nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=0)),
          ('relu5', nn.ReLU()),
          ('conv6', nn.Conv2d(64, 64, kernel_size=2, stride=1, padding=0)),
          ('relu6', nn.ReLU()),
          ('maxpool2', nn.MaxPool2d(2, stride=None)),
    
          ('dropout', nn.Dropout(0.1)),
          ('flatten', nn.Flatten()),
          ('fc1', nn.Linear(256, output_size)),
          # ('tanh', nn.Tanh()),
          # ('fc2', nn.Linear(512, output_size)),
          # ('relu7', nn.ReLU()), 
        ]))
        elif model_key == 1:
            resnet18 = models.resnet18(pretrained=False)
            resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)
            resnet18.fc = nn.Linear(in_features=512, out_features=10, bias=True)
            model = resnet18
        elif model_key == 1.5:
            resnet18 = models.resnet18(pretrained=False)
            resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)
            resnet18.fc = nn.Linear(in_features=512, out_features=10, bias=True)
            model = resnet18
            model.eval()
        elif model_key == 2:
            resnet34 = models.resnet34(pretrained=False)
            resnet34.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)
            resnet34.fc = nn.Linear(in_features=512, out_features=10, bias=True)
            model = resnet34
        elif model_key == 3:
            resnet50 = models.resnet50(pretrained=False)
            resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)
            resnet50.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
            model = resnet50
        elif model_key == 4:
            alexnet = models.alexnet(pretrained=False)
            alexnet.features[0] = nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=2)
            # changed for kernel size and stride, else 0 dimension
            alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=10, bias=True)
            model = alexnet            
        elif model_key == 5:           
            vgg16 = models.vgg16(pretrained=False)
            vgg16.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
            for l in vgg11.features:
                if type(l) == torch.nn.modules.conv.Conv2d:
                    l.kernel_size = (2, 2)
            vgg16.avgpool: nn.AdaptiveAvgPool2d(output_size=(2, 2))
            vgg16.classifier[0] = nn.Linear(in_features=12544, out_features=4096, bias=True)
            vgg16.classifier[6] = nn.Linear(in_features=4096, out_features=10, bias=True)
            vgg16.features = vgg16.features[:11]
            model = vgg16
        elif model_key == 6:
            resnet18 = models.resnet18(pretrained=False)
            resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=2, padding=1,bias=False)
            resnet18.fc = nn.Linear(in_features=512, out_features=10, bias=True)
            model = resnet18
        else:
            print ("model undefined")
        
        if torch.cuda.is_available():
            model = model.cuda()
        
        losses, res_train, res_test= train(model, X_train, y_train,
                                    epochs=epochs,
                                    learning_rate = lr, 
                                    weight_decay = weight_decay,
                                    n_chunks = cur_chunks,
                                    optimizer="SGD")        
        # losses, res_train, res_test, res_time = train(model, X_train, y_train,
        #                             epochs=epochs,
        #                             learning_rate = lr, 
        #                             weight_decay = weight_decay,
        #                             n_chunks = cur_chunks,
        #                             optimizer="SGD", 
        #                                    is_time=True)

        res = pd.DataFrame()
        res['TrainLoss'] = losses
        res['TrainAccuracy'] = res_train
        res['TestAccuracy'] = res_test
        # res['Time'] = res_time

        file_name = model_key_to_name[model_key] + '_' + str(lr) + '_' + str(cur_chunks)
        res.to_csv('results/' + file_name + '_20epochs.csv')

In [ ]:
is_save = False
if is_save:
    torch.save(resnet18.state_dict(), 'results/resnet18_0.1_2000.pt')

# Saliency Map

In [ ]:
def compute_saliency_maps(X, y, model):
    """
    Compute a class saliency map using the model for images X and labels y.

    Input:
    - X: Input images; Tensor of shape (N, 3, H, W)
    - y: Labels for X; LongTensor of shape (N,)
    - model: A pretrained CNN that will be used to compute the saliency map.

    Returns:
    - saliency: A Tensor of shape (N, H, W) giving the saliency maps for the input
    images.
    """
    # Make sure the model is in "test" mode
    model.eval()
    
    # Wrap the input tensors in Variables
    X_var = Variable(X, requires_grad=True)
    y_var = Variable(y)
    saliency = None
    
    scores = model(X_var)
    labels_scores = scores.gather(1, y_var.view(-1, 1)).squeeze()
    
    loss = -torch.sum(torch.log(labels_scores))
    loss.backward()
    
    images_grads = X_var.grad.data
    abs_images_grads = images_grads.abs()
    saliency, _ = abs_images_grads.max(dim=1)

    return saliency


def show_saliency_maps(X, y, path=None, vmax=0.3):
    # Convert X and y from numpy arrays to Torch Tensors
    X_tensor = X#torch.cat([preprocess(Image.fromarray(x)) for x in X], dim=0)
    y_tensor = y

    # Compute saliency maps for images in X
    saliency = compute_saliency_maps(X_tensor, y_tensor, resnet18)

    # Convert the saliency map from Torch Tensor to numpy array and show images
    # and saliency maps together.
    saliency = saliency.cpu().numpy()
    N = X.shape[0]
    for i in range(N):
        plt.subplot(2, N, i + 1)
        plt.imshow(X[i][0].cpu())
        # plt.axis('off')
        # plt.title('Saliency Map')
        plt.subplot(2, N, N + i + 1)
        plt.imshow(saliency[i], cmap=plt.cm.hot, vmin=0, vmax=vmax)
        plt.savefig(path)
        # plt.axis('off')
        plt.gcf().set_size_inches(12, 5)
    plt.show()
    return X

In [ ]:
# for k in y_test[[1, 6, 4, 11]]:
#     print (categories[k])

In [ ]:
X = show_saliency_maps(X_test[[1, 6, 4, 11]], y_test[[1, 6, 4, 11]], path='results/resnet18_saliency_correct.png')

In [ ]:
test_pred_labels = get_labels(resnet18, X_test)

In [ ]:
misclassify_res = pd.DataFrame(columns=['index', 'pred', 'actual'])
for i in range(len(test_pred_labels)):
    if test_pred_labels[i][0] != y_test[i]:
        misclassify_res = misclassify_res.append(
            {'index': i,
             'pred': categories[test_pred_labels[i][0]],
             'actual': categories[y_test[i]] }, 
            ignore_index=True)
print (misclassify_res.head(10))
misclassify_res.to_csv('results/misclassify.csv')

In [ ]:
show_saliency_maps(X_test[[5, 16, 38, 69]], y_test[[5, 16, 38, 69]], path='results/resnet18_saliency_incorrect.png')